In [1]:
import pandas as pd
import numpy as np
import sklearn
import os

讀取資料，並且計算有哪些衛材需要排序

In [10]:
# rooms = ['五病房','六病房','加護病房','呼吸照護病房']
# rooms = ['一病房','二病房','三病房','八病房']

rooms = ['八病房']
year = 2022

need_count = pd.DataFrame()
for room in rooms:
    data = pd.read_csv(f"../使用量計算/全部資料合併/{year}_{room}_合併檔.csv")
    data = data[data.columns.drop(list(data.filter(regex='Unnamed')))]
    # 刪除重複的值
    data = data.drop_duplicates(subset=['料號'], keep='last')
    # 儲存需要計算的值
    temp = data['料號'].reset_index(drop = True)
    need_count = pd.concat([need_count, temp])
need_count.rename(columns={0: '料號'}, inplace=True)
# 刪除重複的值
need_count = need_count.drop_duplicates(subset=['料號'], keep='last').reset_index(drop = True)
print(need_count)

             0
0    A00120017
1    A00120025
2    A00120033
3    A00120059
4    A00120067
..         ...
122  U91700000
123  V00400273
124  V02102000
125  V02204118
126  Y12876111

[127 rows x 1 columns]


排序

In [11]:
# 儲存衛材總週數，和有幾個衛材也是同週數 ex.52週 ，總共有100筆都是52週(週、筆)
type_list = pd.DataFrame()
# 站存四個病房資料
datas_temp = pd.DataFrame()
# 儲存所有衛材
answer = pd.DataFrame()

# type_list.columns = ['週', '筆']
for i in range(len(need_count)):
    part_no = need_count.loc[i:i].reset_index(drop=True).values[0][0]
    print(part_no)

    datas_temp = pd.DataFrame()
    for room in rooms:
#     對四個病房的所有資料加總
#         data_temp = pd.read_csv(f"../使用量計算/以週為單位/2021/{room}/{part_no}_2021週期資料.csv")
        try:
            data_temp = pd.read_csv(f"../使用量計算/以週為單位/2022/{room}/{part_no}_2022週期資料.csv")
        except:
            continue
        
    #     刪除數量為0的週數
        data_temp = data_temp.drop(data_temp[data_temp['數量'] == 0].index)
        datas_temp = pd.concat([datas_temp, data_temp])
    # 刪除重複週的值
    datas_temp = datas_temp.drop_duplicates(subset=['週'], keep='last')

    
#     儲存所有衛材，並以週排序
    value = {"料號":part_no,"紀錄週數":len(datas_temp)}
    answer = answer.append(value, ignore_index=True)
answer = answer.sort_values(by='紀錄週數',ascending=False).reset_index(drop = True)
answer
# answer.to_csv(f"./比對衛材+品名2.csv",encoding='utf_8_sig') 

A00120017
A00120025
A00120033
A00120059
A00120067
A00120102
A00120110
A00120213
A00120504
A00124417
A00421174
A00421190
A01130509
A02120180
A02120225
A02312182
A02320186
A02320340
A02322259
A03010025
A03010041
A03010382
A03295054
A04030052
A04200004
A04200020
A04200038
A04300023
A04411285
A04411370
A04800036
A10700160
A46441986
A46445209
B00204053
B00206057
B00304056
B00306050
B03106859
B03110120
B03600101
B03647700
B04805109
B05569084
B08143445
B08518181
B10106197
B10137087
B13000606
B13000656
B13000709
B13000759
B13330609
B13500651
B13500704
C00041613
C00120168
C00120184
C00120203
C00123169
C00123185
C00123204
C00123220
C00130201
C00130227
C00211075
C00733004
C00733218
C02100500
C02113123
C02513167
C02514164
C02514180
C02813013
C02911226
C03613006
C05110284
C08224755
C09113003
C09113011
C09119005
C09119013
C09133700
C09133750
C13830005
C13930399
C13931431
C13931457
C13941807
C14000004
C14700174
C17200027
C18100201
C18100251
C90330003
D00213208
D00320301
D65221025
F00000021
F00230005


C:\Users\IDSL\AppData\Local\Temp\ipykernel_24316\2754644464.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  answer = answer.append(value, ignore_index=True)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_24316\2754644464.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  answer = answer.append(value, ignore_index=True)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_24316\2754644464.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  answer = answer.append(value, ignore_index=True)
C:\Users\IDSL\AppData\Local\Temp\ipykernel_24316\2754644464.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  answer = answer.append(value, ignore_index=Tr

,料號,紀錄週數
0,A00120017,0
1,C09119005,0
2,C18100251,0
3,C18100201,0
4,C17200027,0
...,...,...
122,B00306050,0
123,B00304056,0
124,B00206057,0
125,B00204053,0


In [12]:
all_Eisai = pd.read_excel(f"../大庫所有衛材品項及實際撥補量資料.xlsx",sheet_name = "大庫房的所有衛材品項資料",index_col=0)
all_Eisai = all_Eisai[all_Eisai.columns.drop(list(all_Eisai.filter(regex='Unnamed')))]
all_Eisai.reset_index(inplace=True)
all_Eisai = all_Eisai.rename(columns = {'Order_Code':'料號'})
all_Eisai['料號'] = all_Eisai['料號'].str.strip()
all_Eisai = all_Eisai[['Chinese_Name','規格','型號','料號']]
data = pd.merge(answer,all_Eisai, on=["料號"],how="inner")

data = data.drop(data[data['紀錄週數'] == 0].index)
data = data.drop_duplicates(subset=['料號'], keep='last')
data.to_csv(f"./病房2022使用量用頻率排序/{rooms}.csv",encoding='utf_8_sig') 
data

OSError: Cannot save file into a non-existent directory: '病房2022使用量用頻率排序'